In [1]:
# module 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

import copy
import random

In [2]:
def select_important_feautre(columns, feature_importance, num):
    
    #딕셔너리
    dic = {}
    for value in [list(v) for v in zip(columns, feature_importance)]:
        dic[value[1]] = value[0] 
        
    sorted_list = sorted(dic)
    sorted_list.reverse()
    
    i = 0
    result = []
    for y in sorted_list:
        result.append(dic[y])
        i = i+1
        if(i == num):
            break
    
    return result

## 데이터셋

In [3]:
x_train_load_fin_v2 = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_data.csv').sort_values(by='acc_id', ascending=True)
y_train_load = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_label.csv').sort_values(by='acc_id', ascending=True)

In [4]:
x_train_fin_v2 = x_train_load_fin_v2
x_train_fin_v2 = x_train_load_fin_v2.drop(['acc_id'], axis=1)
y_data = y_train_load.label

## 피처 리스트

In [5]:
forest_feature = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=30)

In [6]:
forest_feature.fit(x_train_fin_v2, y_data)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [7]:
feature_list = select_important_feautre(x_train_fin_v2.columns, forest_feature.feature_importances_, 754)

In [8]:
x_train_fin_v2 = x_train_fin_v2.loc[:, feature_list]

## cv check - function

In [9]:
forest = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=3, max_depth=30)

In [10]:
def cross_val_score_custom(model, x, y, cv, dev = 0):
    
#     print('\n\n')
    
    result = []
    
    for train_idx, test_idx in cv.split(x):
        
        new_x_train = x.iloc[train_idx, :]
        new_y_train = y.iloc[train_idx]
        
        new_x_test = x.iloc[test_idx, :]
        new_y_test = y.iloc[test_idx]
        
        model.fit(new_x_train, new_y_train)
        
        temp = np.mean(f1_score(new_y_test, model.predict(new_x_test), average=None))
        
        if dev:
            print('dev Mode # : {}'.format(temp))
        
        result.append(temp)
    
#     print('\n\n')
    
    return np.mean(result)

In [11]:
feature_list = feature_list[0:423]

In [12]:
for i in range(0,613):
    
    x_train_fin_v2 = x_train_fin_v2.loc[:, feature_list]
    scores_fin_v2 = cross_val_score_custom(forest, x_train_fin_v2, y_data, KFold(n_splits=5), dev = 0)
    print("feature 개수 : {} => f1 score: {}".format(613-i, np.mean(scores_fin_v2)))
    
    feature_list = feature_list[0:-1]

feature 개수 : 613 => f1 score: 0.7366406496459972
feature 개수 : 612 => f1 score: 0.7366370555127281
feature 개수 : 611 => f1 score: 0.7364759450569487
feature 개수 : 610 => f1 score: 0.7369641982083789
feature 개수 : 609 => f1 score: 0.7359807192811451
feature 개수 : 608 => f1 score: 0.7365923814994599
feature 개수 : 607 => f1 score: 0.7368779420240708
feature 개수 : 606 => f1 score: 0.7363093119987418
feature 개수 : 605 => f1 score: 0.7369196869806778
feature 개수 : 604 => f1 score: 0.7369730827554266
feature 개수 : 603 => f1 score: 0.7370667065586275
feature 개수 : 602 => f1 score: 0.7363802882025198
feature 개수 : 601 => f1 score: 0.7367973558663299
feature 개수 : 600 => f1 score: 0.7363661854975095
feature 개수 : 599 => f1 score: 0.7367159416447073
feature 개수 : 598 => f1 score: 0.7365160150080363
feature 개수 : 597 => f1 score: 0.736213330097285
feature 개수 : 596 => f1 score: 0.736037850088198
feature 개수 : 595 => f1 score: 0.736754014337985
feature 개수 : 594 => f1 score: 0.7363838218835704


KeyboardInterrupt: 

In [ ]:
print(cross_val_score_custom(forest, x_train_fin_v2, y_data, KFold(n_splits=5), dev = 0))